In [1]:
# repo rootz
%cd /work

/work


In [2]:
!ls 

LICENSE    docker    out    pyproject.toml  scripts  tests
README.md  examples  paper  resources	    src


In [3]:
#!chmod +x resources/gene_id_maps/build_entrez_id_map.sh
#! ./resources/gene_id_maps/build_entrez_id_map.sh

In [4]:
#!chmod +x resources/gene_id_maps/build_ensembl_to_symbol_from_gtf.sh
#! ./resources/gene_id_maps/build_ensembl_to_symbol_from_gtf.sh --download

In [5]:
# Fetch from UCSC Xena
!python paper/scripts/fig2_fetch_inputs.py

[fetch_inputs] OK
  /work/paper/source_data/PANCAN_TP53_v1/raw/expression.xena.gz
  /work/paper/source_data/PANCAN_TP53_v1/raw/mc3.v0.2.8.PUBLIC.xena.gz
  /work/paper/source_data/PANCAN_TP53_v1/raw/TCGA_phenotype_dense.tsv.gz


In [5]:
# Extract TP53 mutations (mc3.v0.2.8.PUBLIC.xena.gz)
# Create barcode→cancer from phenotype 
# Output derived/groups/{CANCER}.groups.tsv　for each cancer type

!python paper/scripts/fig2_make_groups.py

[make_groups] sample_type_id raw unique (head): ['07', '06', '05', '03', '02', '01', '11', '']
[make_groups] sample_type_id norm counts: {'1': 10593, '11': 1475, '6': 396, '3': 200, '': 72, '2': 56, '5': 11, '7': 1}
[make_groups] filtered to primary-like sample_type_id in {1,3}: 10793
[make_groups] phenotype mapped samples: 10793
[make_groups] cancers: ['ACC', 'BLCA', 'BRCA', 'CESC', 'CHOL', 'COAD', 'DLBC', 'ESCA', 'GBM', 'HNSC', 'KICH', 'KIRC', 'KIRP', 'LAML', 'LGG', 'LIHC', 'LUAD', 'LUSC', 'MESO', 'OV', 'PAAD', 'PCPG', 'PRAD', 'READ', 'SARC', 'SKCM', 'STAD', 'TGCT', 'THCA', 'THYM', 'UCEC', 'UCS', 'UVM']
[make_groups] MC3 TP53 rows: 3729
[make_groups] MC3 TP53 protein-altering rows (pre-FILTER): 3620
[make_groups] FILTER column not found; not applying FILTER
[make_groups] WARNING: LAML has n_mut=0, n_wt=200
[make_groups] WARNING: UVM has n_mut=0, n_wt=80
[make_groups] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/derived/groups/PANCAN.groups.tsv
  cancers: 33
  samples (mapped): 

In [3]:
!rm -rf /work/paper/source_data/PANCAN_TP53_v1/sample_cards/*
!python paper/scripts/fig2_make_sample_cards.py

[make_sample_cards] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/sample_cards (33 cancers × 2 cards)


In [4]:
!cat /work/paper/source_data/PANCAN_TP53_v1/sample_cards/HNSC.note.sample_card.json

{
  "comparison": "TP53_mut_vs_TP53_wt",
  "condition": "HNSC",
  "extra": {
    "audit_tau": 0.8,
    "benchmark_id": "PANCAN_TP53_v1",
    "context_gate_mode": "note",
    "gene_id_map_tsv": "resources/gene_id_maps/id_map.tsv.gz",
    "goal": "context-conditioned pathway claim selection with calibrated abstention"
  },
  "k_claims": 100,
  "notes": "TCGA Pan-Cancer; groups defined from MC3 TP53 PASS mutations.",
  "perturbation": "genotype",
  "tissue": "tumor"
}


In [5]:
!cat /work/paper/source_data/PANCAN_TP53_v1/sample_cards/HNSC.hard.sample_card.json

{
  "comparison": "TP53_mut_vs_TP53_wt",
  "condition": "HNSC",
  "extra": {
    "audit_tau": 0.8,
    "benchmark_id": "PANCAN_TP53_v1",
    "context_gate_mode": "hard",
    "gene_id_map_tsv": "resources/gene_id_maps/id_map.tsv.gz",
    "goal": "context-conditioned pathway claim selection with calibrated abstention"
  },
  "k_claims": 100,
  "notes": "TCGA Pan-Cancer; groups defined from MC3 TP53 PASS mutations.",
  "perturbation": "genotype",
  "tissue": "tumor"
}


In [10]:
for c in ("HNSC", "LUAD", "LUSC", "BRCA", "OV",  "UCEC",  "SKCM"):
  !Rscript paper/scripts/fig2_deg_rank.R $c


[deg_rank] inputs
  cancer: HNSC 
  expr: /work/paper/source_data/PANCAN_TP53_v1/raw/expression.xena.gz 
  groups: /work/paper/source_data/PANCAN_TP53_v1/derived/groups/HNSC.groups.tsv 
[deg_rank] matched samples: 520  / expr= 11069  / groups= 528 
[deg_rank] QC: dropping zero-variance genes: 269 
[deg_rank] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/HNSC.deg_ranking.tsv 
  n_genes: 20262 
[deg_rank] inputs
  cancer: LUAD 
  expr: /work/paper/source_data/PANCAN_TP53_v1/raw/expression.xena.gz 
  groups: /work/paper/source_data/PANCAN_TP53_v1/derived/groups/LUAD.groups.tsv 
[deg_rank] matched samples: 515  / expr= 11069  / groups= 520 
[deg_rank] QC: dropping zero-variance genes: 338 
[deg_rank] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/LUAD.deg_ranking.tsv 
  n_genes: 20193 
[deg_rank] inputs
  cancer: LUSC 
  expr: /work/paper/source_data/PANCAN_TP53_v1/raw/expression.xena.gz 
  groups: /work/paper/source_data/PANCAN_TP53_v1/derived/gr

In [6]:
# Hallmark
for c in ("HNSC", "LUAD", "LUSC", "BRCA", "OV",  "UCEC",  "SKCM"):
  !Rscript paper/scripts/figSx_fgsea_to_evidence_table.R $c --collection H


[fgsea_to_evidence] inputs
  cancer: HNSC 
  rank: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/HNSC.deg_ranking.tsv 
  out : /work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.H.evidence_table.tsv 
  collection: H 
  subcategory: <none> 
  source_tag: fgsea_msigdb_H_entrez 
  msigdbr_version: 25.1.1 
  fgsea_version  : 1.36.2 
Warning message:
The `category` argument of `msigdbr()` is deprecated as of msigdbr 10.0.0.
ℹ Please use the `collection` argument instead. 
[fgsea_to_evidence] msigdbr entrez column: entrez_gene 
[fgsea_to_evidence] overlap diagnostics
  overlap_summary: min 24 p25 80 median 141 p75 165 max 193 
  pathways_with_overlap>=15: 50 / 50 
[fgsea_to_evidence] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.H.evidence_table.tsv 
  n_terms: 50 
[fgsea_to_evidence] inputs
  cancer: LUAD 
  rank: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/LUAD.deg_ranking.tsv 
  out : /work/paper/source_data/PANCAN_TP53_v1/evidence_

In [7]:
# GO
for c in ("HNSC", "LUAD", "LUSC", "BRCA", "OV",  "UCEC",  "SKCM"):
   !Rscript paper/scripts/figSx_fgsea_to_evidence_table.R $c --collection C5 --subcategory GO:BP


[fgsea_to_evidence] inputs
  cancer: HNSC 
  rank: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/HNSC.deg_ranking.tsv 
  out : /work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.C5_GO_BP.evidence_table.tsv 
  collection: C5 
  subcategory: GO:BP 
  source_tag: fgsea_msigdb_C5_GO_BP_entrez 
  msigdbr_version: 25.1.1 
  fgsea_version  : 1.36.2 
Warning message:
The `category` argument of `msigdbr()` is deprecated as of msigdbr 10.0.0.
ℹ Please use the `collection` argument instead. 
[fgsea_to_evidence] msigdbr entrez column: entrez_gene 
[fgsea_to_evidence] overlap diagnostics
  overlap_summary: min 0 p25 6 median 13 p75 38 max 1258 
  pathways_with_overlap>=15: 3525 / 7583 
[fgsea_to_evidence] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.C5_GO_BP.evidence_table.tsv 
  n_terms: 3386 
[fgsea_to_evidence] inputs
  cancer: LUAD 
  rank: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/LUAD.deg_ranking.tsv 
  out : /work/paper/source_data/

In [12]:
# Reactome / KEGG
for c in ("HNSC", "LUAD", "LUSC", "BRCA", "OV",  "UCEC",  "SKCM"):
  !Rscript paper/scripts/figSx_fgsea_to_evidence_table.R $c --collection C2 --subcategory CP:REACTOME
  !Rscript paper/scripts/figSx_fgsea_to_evidence_table.R $c --collection C2 --subcategory CP:KEGG_MEDICUS



[fgsea_to_evidence] inputs
  cancer: HNSC 
  rank: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/HNSC.deg_ranking.tsv 
  out : /work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.C2_CP_REACTOME.evidence_table.tsv 
  collection: C2 
  subcategory: CP:REACTOME 
  source_tag: fgsea_msigdb_C2_CP_REACTOME_entrez 
  msigdbr_version: 25.1.1 
  fgsea_version  : 1.36.2 
Warning message:
The `category` argument of `msigdbr()` is deprecated as of msigdbr 10.0.0.
ℹ Please use the `collection` argument instead. 
[fgsea_to_evidence] msigdbr entrez column: entrez_gene 
[fgsea_to_evidence] overlap diagnostics
  overlap_summary: min 0 p25 9 median 19 p75 45 max 922 
  pathways_with_overlap>=15: 1036 / 1787 
[fgsea_to_evidence] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.C2_CP_REACTOME.evidence_table.tsv 
  n_terms: 984 
[fgsea_to_evidence] inputs
  cancer: HNSC 
  rank: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/HNSC.deg_ranking.tsv 
  out : /w

In [64]:
# Fig2 run
!python paper/scripts/run_figSx_collections_pipeline.py \
  --benchmark-id PANCAN_TP53_v1 \
  --cancers HNSC,LUAD,LUSC,BRCA,OV,UCEC,SKCM \
  --collections Hallmark,GO_BP,Reactome,KEGG_MEDICUS \
  --variants ours \
  --gate-modes hard \
  --tau 0.2 \
  --k-claims 50 \
  --out-root "/work/paper/source_data/PANCAN_TP53_v1/out_supp_collections_tau_0.20" \
  --force


[run_figSx] plan
  benchmark_id: PANCAN_TP53_v1
  tau: 0.20
  cancers: HNSC,LUAD,LUSC,BRCA,OV,UCEC,SKCM
  collections: Hallmark,GO_BP,Reactome,KEGG_MEDICUS
  variants: ours
  gate_modes: hard
  k_claims: 50
  seed: 42
  out_root: /work/paper/source_data/PANCAN_TP53_v1/out_supp_collections_tau_0.20
[run_figSx] jobs: 28
[run_figSx] (1/28) Hallmark HNSC ours gate=hard tau=0.20
[run_figSx] (2/28) Hallmark LUAD ours gate=hard tau=0.20
[run_figSx] (3/28) Hallmark LUSC ours gate=hard tau=0.20
[run_figSx] (4/28) Hallmark BRCA ours gate=hard tau=0.20
[run_figSx] (5/28) Hallmark OV ours gate=hard tau=0.20
[run_figSx] (6/28) Hallmark UCEC ours gate=hard tau=0.20
[run_figSx] (7/28) Hallmark SKCM ours gate=hard tau=0.20
[run_figSx] (8/28) GO_BP HNSC ours gate=hard tau=0.20
[run_figSx] (9/28) GO_BP LUAD ours gate=hard tau=0.20
[run_figSx] (10/28) GO_BP LUSC ours gate=hard tau=0.20
[run_figSx] (11/28) GO_BP BRCA ours gate=hard tau=0.20
[run_figSx] (12/28) GO_BP OV ours gate=hard tau=0.20
[run_figSx] 

In [83]:
!python paper/scripts/figSx_collect_collection_metrics.py \
  --out-root /work/paper/source_data/PANCAN_TP53_v1/out_supp_collections_tau_0.20 \
  --benchmark-id PANCAN_TP53_v1 \
  --conditions HNSC,LUAD,LUSC,BRCA,OV,UCEC,SKCM \
  --variants ours \
  --gate-modes hard \
  --taus 0.2


WROTE:
   /work/paper/source_data/PANCAN_TP53_v1/collection_metrics/collections_summary_wide.tsv
   /work/paper/source_data/PANCAN_TP53_v1/collection_metrics/module_survival_long.tsv
   /work/paper/source_data/PANCAN_TP53_v1/collection_metrics/audit_reasons_long.tsv


In [66]:
!ls  /work/paper/source_data/BEATAML_TP53_v1/out/BEATAML/ours/gate_hard/tau_0.82

ls: cannot access '/work/paper/source_data/BEATAML_TP53_v1/out/BEATAML/ours/gate_hard/tau_0.82': No such file or directory


In [95]:
!python paper/scripts/figSx_plot_collection_panels.py \
  --collections-summary-wide /work/paper/source_data/PANCAN_TP53_v1/collection_metrics/collections_summary_wide.tsv \
  --module-survival-long     /work/paper/source_data/PANCAN_TP53_v1/collection_metrics/module_survival_long.tsv \
  --audit-reasons-long       /work/paper/source_data/PANCAN_TP53_v1/collection_metrics/audit_reasons_long.tsv \
  --outdir /work/paper/source_data/PANCAN_TP53_v1/fig/SuppFigSx_tau0.20 \
  --fmt pdf \
  --conditions HNSC,LUAD,LUSC,BRCA,OV,UCEC,SKCM \
  --variants ours \
  --gate-modes hard \
  --taus 0.2 \
  --include-fail-reasons


[figSx_plot] WROTE:
  /work/paper/source_data/PANCAN_TP53_v1/fig/SuppFigSx_tau0.20/FigSx_PanelA_module_survival.pdf
  /work/paper/source_data/PANCAN_TP53_v1/fig/SuppFigSx_tau0.20/FigSx_PanelB_module_stats.pdf
  /work/paper/source_data/PANCAN_TP53_v1/fig/SuppFigSx_tau0.20/FigSx_PanelC_outcomes.pdf
  /work/paper/source_data/PANCAN_TP53_v1/fig/SuppFigSx_tau0.20/FigSx_PanelD_top_reasons.pdf


In [85]:
!python paper/scripts/figSx_plot_collection_panels.py \
  --collections-summary-wide /work/paper/source_data/PANCAN_TP53_v1/collection_metrics/collections_summary_wide.tsv \
  --module-survival-long     /work/paper/source_data/PANCAN_TP53_v1/collection_metrics/module_survival_long.tsv \
  --audit-reasons-long       /work/paper/source_data/PANCAN_TP53_v1/collection_metrics/audit_reasons_long.tsv \
  --outdir /work/paper/source_data/PANCAN_TP53_v1/fig/SuppFigSx_tau0.20 \
  --fmt pdf \
  --conditions HNSC,LUAD,LUSC,BRCA,OV,UCEC,SKCM \
  --variants ours \
  --gate-modes hard \
  --taus 0.2 \
  --include-fail-reasons


/work/paper/scripts/figSx_plot_collection_panels.py:183: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  bp = ax.boxplot(
[figSx_plot] WROTE:
  /work/paper/source_data/PANCAN_TP53_v1/fig/SuppFigSx_tau0.20/FigSx_PanelA_module_survival.pdf
  /work/paper/source_data/PANCAN_TP53_v1/fig/SuppFigSx_tau0.20/FigSx_PanelB_module_stats.pdf
  /work/paper/source_data/PANCAN_TP53_v1/fig/SuppFigSx_tau0.20/FigSx_PanelC_outcomes.pdf
  /work/paper/source_data/PANCAN_TP53_v1/fig/SuppFigSx_tau0.20/FigSx_PanelD_top_reasons.pdf


In [20]:
import pandas as pd
df = pd.read_csv("/work/paper/source_data/PANCAN_TP53_v1/out/risk_coverage.tsv", sep="\t")
g = df.groupby(["condition","variant"])["tau"].nunique().sort_values()
print(g.head(20))
print("min n_tau =", g.min(), "max n_tau =", g.max(), "rows =", len(df))

condition  variant     
BRCA       context_swap    5
UCEC       context_swap    5
SKCM       stress          5
           ours            5
           context_swap    5
OV         stress          5
           ours            5
           context_swap    5
LUSC       stress          5
UCEC       ours            5
LUSC       ours            5
LUAD       stress          5
           ours            5
           context_swap    5
HNSC       stress          5
           ours            5
           context_swap    5
BRCA       stress          5
           ours            5
LUSC       context_swap    5
Name: tau, dtype: int64
min n_tau = 5 max n_tau = 5 rows = 105


In [81]:
!python paper/scripts/fig2_plot_multipanel.py \
  --in  /work/paper/source_data/PANCAN_TP53_v1/out/risk_coverage.tsv \
  --out /work/paper/source_data/PANCAN_TP53_v1/fig/Fig2_Supp_multipanel_abstain_variants.pdf \
  --ycol abstain_rate_total \
  --variants ours,context_swap,stress \
  --label-mode variant


In [75]:
!python paper/scripts/fig2_lines_status_by_tau.py \
  --in /work/paper/source_data/PANCAN_TP53_v1/out/risk_coverage.tsv \
  --out /work/paper/source_data/PANCAN_TP53_v1/fig/FIg2_HNSC_abstain_vs_tau_variants.pdf \
  --condition HNSC \
  --gate-mode hard \
  --variants ours,context_swap,stress \
  --metric abstain \
  --label-mode proposed


In [76]:
!python paper/scripts/fig2_make_labels_template.py \
  --root /work/paper/source_data/PANCAN_TP53_v1/out \
  --out  /work/paper/source_data/PANCAN_TP53_v1/labels/labels_hnsc_R1_v1.tsv \
  --condition HNSC \
  --per-run 30 \
  --prefer-status PASS \
  --with-hints \
  --rater-id R1 \
  --label-version v1


[OK] wrote: /work/paper/source_data/PANCAN_TP53_v1/labels/labels_hnsc_R1_v1.tsv rows=49
[NEXT] Fill human_label with: ACCEPT / REJECT / SHOULD_ABSTAIN (partial labels ok)


In [55]:
!python paper/scripts/fig2_check_labels_merge.py \
  --labels /work/paper/source_data/PANCAN_TP53_v1/labels/labels_hnsc_R1_v1.tsv \
  --root   /work/paper/source_data/PANCAN_TP53_v1/out \
  --condition HNSC \
  --prefer-status PASS


[OK] labels claim_id unique
[OK] labels human_label values look valid (or blank)
[OK] audit logs used: 15
[OK] unique claim_id in audit (all statuses): 50
[OK] unique claim_id in audit (status=PASS): 43
[OK] labels rows: 49
[OK] labels claim_id found in audit: 49
[OK] non-blank labels total: 49
[OK] non-blank labels among audit status=PASS: 43
[INFO] fraction of non-blank labels that target status=PASS: 0.878
[DONE] If this is clean, pass --labels to fig2_collect_risk_coverage.py


In [58]:
!python paper/scripts/fig2_collect_risk_coverage.py \
  --root /work/paper/source_data/PANCAN_TP53_v1/out \
  --out  /work/paper/source_data/PANCAN_TP53_v1/out/risk_coverage.label.tsv \
  --method ours \
  --labels /work/paper/source_data/PANCAN_TP53_v1/labels/labels_hnsc_R1_v1.tsv


[OK] wrote: /work/paper/source_data/PANCAN_TP53_v1/out/risk_coverage.label.tsv rows=105


In [84]:
!python paper/scripts/fig2_plot_scatter_human_risk.py \
  --in  /work/paper/source_data/PANCAN_TP53_v1/out/risk_coverage.label.tsv \
  --out /work/paper/source_data/PANCAN_TP53_v1/fig/Fig2_HNSC_scatter_human_risk.pdf \
  --condition HNSC \
  --ycol risk_human_nonaccept \
  --variants ours,context_swap,stress \
  --annotate-tau \
  --annotate-npass


In [5]:
!ls /work/paper/source_data/PANCAN_TP53_v1/out/HNSC

context_swap  ours  stress


In [39]:
!LLMPATH_BACKEND=ollama \
LLMPATH_OLLAMA_HOST=http://ollama:11434 \
LLMPATH_OLLAMA_MODEL=llama3.1:8b \
LLMPATH_CONTEXT_REVIEW_MODE=llm \
LLMPATH_CONTEXT_GATE_MODE=hard \
LLMPATH_DEBUG=1 \
LLMPATH_CLAIM_MODE=llm \
python paper/scripts/run_fig2_pipeline.py \
  --cancers HNSC \
  --variants ours \
  --gate-modes hard \
  --taus 0.8 \
  --k-claims 50 \
  --context-review-mode llm \
  --out-root /work/paper/source_data/PANCAN_TP53_v1/out_llm \
  --force


[run_fig2] plan
  benchmark_id: PANCAN_TP53_v1
  conditions: HNSC
  variants: ours
  gate_modes: hard
  taus: 0.80
  k_claims: 50
  seed: 42
  out_root: /work/paper/source_data/PANCAN_TP53_v1/out_llm
[run_fig2] jobs: 1
[run_fig2] (1/1) HNSC ours gate=hard tau=0.80
[select] mode_eff=llm backend=yes review_backend=yes k=50 ctx_review_mode=llm
[context_review] updated=50 shortlist=50 missing=50
[run_fig2] OK
  out_root: /work/paper/source_data/PANCAN_TP53_v1/out_llm


In [40]:
import json, pathlib
p = pathlib.Path(f"{RUN}/llm_claims.raw.json")
obj = json.loads(p.read_text())
print("keys:", list(obj.keys()))
for k,v in obj.items():
    if isinstance(v,str):
        print(k, "str_len", len(v))
    else:
        print(k, type(v))

import json, re, pathlib
p = pathlib.Path(f"{RUN}/llm_claims.prompt.json")
obj = json.loads(p.read_text())
prompt = obj.get("prompt","")
cand = prompt.split("Candidates (each line has uid/id/name/dir/surv/genes):")[-1]
lines = [ln for ln in cand.splitlines() if ln.strip().startswith("uid=")]
print("n_candidate_lines:", len(lines))

keys: ['raw']
raw str_len 4130
n_candidate_lines: 50


In [41]:
!ls /work/paper/source_data/PANCAN_TP53_v1/out_llm/HNSC/ours/gate_hard/tau_0.80

audit_log.tsv				modules.tsv
claims.proposed.tsv			report.jsonl
context_review_cache.e6cee1e46cac.json	report.md
distilled.tsv				risk_coverage.tsv
distilled.with_modules.tsv		run_meta.json
evidence.normalized.tsv			run_meta.runner.json
llm_claims.candidates.json		sample_card.normalized.json
llm_claims.meta.json			sample_card.raw.json
llm_claims.prompt.json			term_gene_edges.tsv
llm_claims.raw.json			term_modules.tsv
manifest.json


In [65]:
!python paper/scripts/figS_make_llm_examples.py \
  --run-dir /work/paper/source_data/PANCAN_TP53_v1/out_llm/HNSC/ours/gate_hard/tau_0.80 \
  --evidence-table /work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.evidence_table.tsv \
  --out-examples-dir /work/paper/source_data/PANCAN_TP53_v1/llm_examples \
  --n-evidence-rows 8


[figS1_make_examples] wrote bundles:
  - /work/paper/source_data/PANCAN_TP53_v1/llm_examples/examples/claim.proposed.fallback_proxy.json
  - /work/paper/source_data/PANCAN_TP53_v1/llm_examples/examples/context_review.fallback_proxy.json
  - /work/paper/source_data/PANCAN_TP53_v1/llm_examples/examples/audit_decision.fallback_proxy.json
  - /work/paper/source_data/PANCAN_TP53_v1/llm_examples/examples/evidence_ref.fallback_proxy.tsv
  - /work/paper/source_data/PANCAN_TP53_v1/llm_examples/examples/claim.proposed.llm_review.json
  - /work/paper/source_data/PANCAN_TP53_v1/llm_examples/examples/context_review.llm_review.json
  - /work/paper/source_data/PANCAN_TP53_v1/llm_examples/examples/audit_decision.llm_review.json
  - /work/paper/source_data/PANCAN_TP53_v1/llm_examples/examples/evidence_ref.llm_review.tsv


In [66]:
!ls /work/paper/source_data/PANCAN_TP53_v1/llm_examples/examples

audit_decision.fallback_proxy.json  context_review.fallback_proxy.json
audit_decision.llm_review.json	    context_review.llm_review.json
claim.proposed.fallback_proxy.json  evidence_ref.fallback_proxy.tsv
claim.proposed.llm_review.json	    evidence_ref.llm_review.tsv


In [67]:
!python paper/scripts/fig2_make_labels_template.py \
  --root /work/paper/source_data/PANCAN_TP53_v1/out_llm \
  --out /work/paper/source_data/PANCAN_TP53_v1/out_llm/labels/labels_llm_hnsc_R1_v1.tsv \
  --condition HNSC \
  --per-run 30 \
  --prefer-status PASS \
  --with-hints \
  --rater-id R1 \
  --label-version v1


[OK] wrote: /work/paper/source_data/PANCAN_TP53_v1/out_llm/labels/labels_llm_hnsc_R1_v1.tsv rows=20
[NEXT] Fill human_label with: ACCEPT / REJECT / SHOULD_ABSTAIN (partial labels ok)


In [68]:
!python paper/scripts/fig2_check_labels_merge.py \
  --labels /work/paper/source_data/PANCAN_TP53_v1/out_llm/labels/labels_llm_hnsc_R1_v1.20260130.tsv \
  --root   /work/paper/source_data/PANCAN_TP53_v1/out_llm \
  --condition HNSC \
  --prefer-status PASS


[OK] labels claim_id unique
[OK] labels human_label values look valid (or blank)
[OK] audit logs used: 1
[OK] unique claim_id in audit (all statuses): 20
[OK] unique claim_id in audit (status=PASS): 11
[OK] labels rows: 20
[OK] labels claim_id found in audit: 20
[OK] non-blank labels total: 20
[OK] non-blank labels among audit status=PASS: 11
[INFO] fraction of non-blank labels that target status=PASS: 0.550
[DONE] If this is clean, pass --labels to fig2_collect_risk_coverage.py


In [69]:
!python paper/scripts/fig2_collect_risk_coverage.py \
  --root /work/paper/source_data/PANCAN_TP53_v1/out_llm \
  --out  /work/paper/source_data/PANCAN_TP53_v1/out_llm/risk_coverage.label.tsv \
  --method ours \
  --labels /work/paper/source_data/PANCAN_TP53_v1/out_llm/labels/labels_llm_hnsc_R1_v1.20260130.tsv 


[OK] wrote: /work/paper/source_data/PANCAN_TP53_v1/out_llm/risk_coverage.label.tsv rows=1


In [80]:
import pandas as pd

base = "/work/paper/source_data/PANCAN_TP53_v1/out/risk_coverage.label.tsv"
llm  = "/work/paper/source_data/PANCAN_TP53_v1/out_llm/risk_coverage.label.tsv"
out  = "/work/paper/source_data/PANCAN_TP53_v1/out_llm/risk_coverage.label.with_llm.tsv"

df_base = pd.read_csv(base, sep="\t")
df_llm  = pd.read_csv(llm,  sep="\t")

mask = (df_llm.get("method","").astype(str).str.lower() == "ours") & (df_llm["variant"].astype(str).str.lower() == "ours")
df_llm.loc[mask, "variant"] = "ours_llm"
#df_llm.loc[mask, "method"] = "ours_llm"

df = pd.concat([df_base, df_llm], ignore_index=True)
df.to_csv(out, sep="\t", index=False)
print("WROTE:", out, "rows=", len(df))

WROTE: /work/paper/source_data/PANCAN_TP53_v1/out_llm/risk_coverage.label.with_llm.tsv rows= 106


In [81]:
df

,benchmark_id,condition,method,variant,gate_mode,tau,coverage_pass,abstain_rate_total,fail_rate_total,fail_rate_answered,n_total,n_pass,n_abstain,n_fail,n_answered,risk_human_reject,risk_human_nonaccept,n_pass_labeled,risk_proxy,_src
0,NaN,BRCA,ours,context_swap,hard,0.2,0.32,0.68,0.00,0.0000,50.0,16.0,34.0,0.0,16.0,NaN,NaN,0.0,0.0,/work/paper/source_data/PANCAN_TP53_v1/out/BRC...
1,NaN,BRCA,ours,context_swap,hard,0.4,0.32,0.68,0.00,0.0000,50.0,16.0,34.0,0.0,16.0,NaN,NaN,0.0,0.0,/work/paper/source_data/PANCAN_TP53_v1/out/BRC...
2,NaN,BRCA,ours,context_swap,hard,0.6,0.32,0.68,0.00,0.0000,50.0,16.0,34.0,0.0,16.0,NaN,NaN,0.0,0.0,/work/paper/source_data/PANCAN_TP53_v1/out/BRC...
3,NaN,BRCA,ours,context_swap,hard,0.8,0.32,0.68,0.00,0.0000,50.0,16.0,34.0,0.0,16.0,NaN,NaN,0.0,0.0,/work/paper/source_data/PANCAN_TP53_v1/out/BRC...
4,NaN,BRCA,ours,context_swap,hard,0.9,0.28,0.72,0.00,0.0000,50.0,14.0,36.0,0.0,14.0,NaN,NaN,0.0,0.0,/work/paper/source_data/PANCAN_TP53_v1/out/BRC...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,NaN,UCEC,ours,stress,hard,0.4,0.24,0.76,0.00,0.0000,50.0,12.0,38.0,0.0,12.0,NaN,NaN,0.0,0.0,/work/paper/source_data/PANCAN_TP53_v1/out/UCE...
102,NaN,UCEC,ours,stress,hard,0.6,0.24,0.76,0.00,0.0000,50.0,12.0,38.0,0.0,12.0,NaN,NaN,0.0,0.0,/work/paper/source_data/PANCAN_TP53_v1/out/UCE...
103,NaN,UCEC,ours,stress,hard,0.8,0.24,0.76,0.00,0.0000,50.0,12.0,38.0,0.0,12.0,NaN,NaN,0.0,0.0,/work/paper/source_data/PANCAN_TP53_v1/out/UCE...
104,NaN,UCEC,ours,stress,hard,0.9,0.20,0.80,0.00,0.0000,50.0,10.0,40.0,0.0,10.0,NaN,NaN,0.0,0.0,/work/paper/source_data/PANCAN_TP53_v1/out/UCE...


In [83]:
!python paper/scripts/fig2_plot_scatter_human_risk.py \
  --in  /work/paper/source_data/PANCAN_TP53_v1/out_llm/risk_coverage.label.with_llm.tsv \
  --out /work/paper/source_data/PANCAN_TP53_v1/out_llm/fig/Fig2_HNSC_scatter_human_risk_with_llm.pdf \
  --condition HNSC \
  --ycol risk_human_nonaccept \
  --variants ours,ours_llm,context_swap,stress \
  --annotate-tau \
  --annotate-npass


In [87]:
!python paper/scripts/figS_plot_abstain_reasons.py \
  --audit-logs \
    /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/audit_log.tsv \
    /work/paper/source_data/PANCAN_TP53_v1/out_llm/HNSC/ours/gate_hard/tau_0.80/audit_log.tsv \
    /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/context_swap/gate_hard/tau_0.80/audit_log.tsv \
    /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/stress/gate_hard/tau_0.80/audit_log.tsv \
  --out /work/paper/source_data/PANCAN_TP53_v1/out_llm/fig/FigS_HNSC_abstain_reasons_tau0.80_with_llm.pdf \
  --title "" \
  --topk 5


In [89]:
!python paper/scripts/figS_plot_fail_reasons.py \
  --audit-logs \
    /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/audit_log.tsv \
    /work/paper/source_data/PANCAN_TP53_v1/out_llm/HNSC/ours/gate_hard/tau_0.80/audit_log.tsv \
    /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/context_swap/gate_hard/tau_0.80/audit_log.tsv \
    /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/stress/gate_hard/tau_0.80/audit_log.tsv \
  --out /work/paper/source_data/PANCAN_TP53_v1/out_llm/fig/FigS_HNSC_fail_reasons_tau0.80_with_llm.pdf \
  --title "" \
  --topk 5
